In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()

In [2]:
train = pd.read_csv("/Users/Matt/Documents/GitHub/kaggle-projects/toxic_comments/train.csv")
test = pd.read_csv("/Users/Matt/Documents/GitHub/kaggle-projects/toxic_comments/test.csv")
test_labels = pd.read_csv("/Users/Matt/Documents/GitHub/kaggle-projects/toxic_comments/test_labels.csv")

In [3]:
test_labels_scored = test_labels[test_labels['toxic'] != -1]

In [4]:
test = test.merge(test_labels_scored,how='inner',on='id')

In [5]:
import nltk
from nltk.corpus import stopwords
import string

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

import time


In [6]:
def text_process(mess):
    '''
    1. remove punctuation
    2. remove stop words
    3. return list of relevant words
    '''
    
    nopunc = [char for char in mess if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return [word.lower() for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [14]:
# clean up and vectorize train and test

start_time = time.time()

bow_transformer = CountVectorizer(analyzer=text_process).fit(train['comment_text'])
bow_comment_text_train = bow_transformer.transform(train['comment_text'])
bow_comment_text_test = bow_transformer.transform(test['comment_text'])

elapsed_time = time.time() - start_time

print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))
print(len(bow_transformer.vocabulary_))


/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:10: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


06:07:26
257677


In [15]:
# 10,000 observations took 5 minutes
# 20,000 observations took 10:59 minutes
# 223,549 observations (159,571 train, 63,978 test) took 6 hours, 7 minutes, 26 seconds

In [16]:
# tfidf transform train and test

start_time = time.time()

tfidf_transformer = TfidfTransformer().fit(bow_comment_text_train)
tfidf_comment_text_train = tfidf_transformer.transform(bow_comment_text_train) 
tfidf_comment_text_test = tfidf_transformer.transform(bow_comment_text_test)

elapsed_time = time.time() - start_time

print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))


00:00:00


In [17]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


In [18]:
# model

RF = RandomForestClassifier(n_estimators=300)


In [19]:
# fit model - TOXIC

start_time = time.time()

RF.fit(tfidf_comment_text_train,train['toxic'])

elapsed_time = time.time() - start_time

print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))


04:57:07


In [20]:
# 4 hours, 57 minutes, 7 seconds

In [26]:
# make predictions, evaluate - TOXIC

start_time = time.time()

predictions = RF.predict(tfidf_comment_text_test)

elapsed_time = time.time() - start_time

print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))
print(confusion_matrix(test['toxic'],predictions))

# top left: TRUE negatives
# bottom left: FALSE negatives 
# top right: FALSE positives
# bottom right: TRUE positives
# bottom row is the number of positives

01:01:44
[[55889  1999]
 [ 1981  4109]]


In [27]:
# 1 hour, 1 minute, 44 seconds

In [ ]:
# if i had predicted 0 for every observation i would have gotten 6090 wrong
# with my model i got 3980 wrong